In [1]:
# Import dependencies
import pandas as pd
from pathlib import Path
import re
import numpy as np

# Import Data

In [2]:
# Base datasets
squirrel_2020 = Path("Resources/squirrel_data_2020.csv")
squirrel_2018 = Path("Resources/squirrel_data_2018.csv")

# Additional dataset from 2020
parks_2020 = Path("Resources/park_data_2020.csv")

# 2020 Dataset - Squirrels

In [3]:
# Create dataframe from CSV
s2020_df = pd.read_csv(squirrel_2020, encoding="unicode_escape")

# Display DataFrame and its shape
print(f"2020 Squirrel Dataset: {s2020_df.shape}")
s2020_df.head()

2020 Squirrel Dataset: (433, 16)


Area Name Area ID        Park Name  Park ID Squirrel ID  \
0  UPPER MANHATTAN       A  Fort Tryon Park        1     A-01-01   
1  UPPER MANHATTAN       A  Fort Tryon Park        1     A-01-02   
2  UPPER MANHATTAN       A  Fort Tryon Park        1     A-01-03   
3  UPPER MANHATTAN       A  Fort Tryon Park        1     A-01-04   
4  UPPER MANHATTAN       A  Fort Tryon Park        1     A-01-05   

  Primary Fur Color Highlights in Fur Color Color Notes      Location  \
0              Gray                   White         NaN  Ground Plane   
1              Gray                   White         NaN  Ground Plane   
2              Gray                   White         NaN  Ground Plane   
3              Gray                   White         NaN  Ground Plane   
4              Gray                Cinnamon         NaN  Ground Plane   

  Above Ground (Height in Feet) Specific Location                 Activities  \
0                           NaN               NaN                   Foraging   
1                           NaN               NaN                   Foraging   
2                           NaN               NaN  Eating, Digging something   
3                           NaN               NaN                    Running   
4                           NaN               NaN            Running, Eating   

  Interactions with Humans Other Notes or Observations  \
0              Indifferent                         NaN   
1              Indifferent                Looks skinny   
2              Indifferent                         NaN   
3              Indifferent                         NaN   
4              Indifferent               She left food   

   Squirrel Latitude (DD.DDDDDD)  Squirrel Longitude (-DD.DDDDDD)  
0                      40.859410                       -73.933936  
1                      40.859436                       -73.933937  
2                      40.859416                       -73.933894  
3                      40.859418                       -73.933895  
4                      40.859493                       -73.933590

In [4]:
# Drop null values for minimum requirement columns
dropna_columns = [
    "Primary Fur Color",
    "Highlights in Fur Color",
    "Activities",
    "Interactions with Humans",
    "Squirrel Latitude (DD.DDDDDD)",
    "Squirrel Longitude (-DD.DDDDDD)"
]

s2020_nonull = s2020_df.dropna(subset=dropna_columns, how="any")

# Display DataFrame and its shape
print(f"{s2020_nonull.shape}")
s2020_nonull.head()

(192, 16)


Area Name Area ID        Park Name  Park ID Squirrel ID  \
0  UPPER MANHATTAN       A  Fort Tryon Park        1     A-01-01   
1  UPPER MANHATTAN       A  Fort Tryon Park        1     A-01-02   
2  UPPER MANHATTAN       A  Fort Tryon Park        1     A-01-03   
3  UPPER MANHATTAN       A  Fort Tryon Park        1     A-01-04   
4  UPPER MANHATTAN       A  Fort Tryon Park        1     A-01-05   

  Primary Fur Color Highlights in Fur Color Color Notes      Location  \
0              Gray                   White         NaN  Ground Plane   
1              Gray                   White         NaN  Ground Plane   
2              Gray                   White         NaN  Ground Plane   
3              Gray                   White         NaN  Ground Plane   
4              Gray                Cinnamon         NaN  Ground Plane   

  Above Ground (Height in Feet) Specific Location                 Activities  \
0                           NaN               NaN                   Foraging   
1                           NaN               NaN                   Foraging   
2                           NaN               NaN  Eating, Digging something   
3                           NaN               NaN                    Running   
4                           NaN               NaN            Running, Eating   

  Interactions with Humans Other Notes or Observations  \
0              Indifferent                         NaN   
1              Indifferent                Looks skinny   
2              Indifferent                         NaN   
3              Indifferent                         NaN   
4              Indifferent               She left food   

   Squirrel Latitude (DD.DDDDDD)  Squirrel Longitude (-DD.DDDDDD)  
0                      40.859410                       -73.933936  
1                      40.859436                       -73.933937  
2                      40.859416                       -73.933894  
3                      40.859418                       -73.933895  
4                      40.859493                       -73.933590

## Columns Cleaning
- Remove columns (2020):
    - 'Area Name', irrelevant.
    - 'Area ID, irrelevant.
    - 'Color Notes', irrelevant and too few data points.
    - 'Location', irrelevant and would take too much resources to clean the data. ***
    - 'Above Ground (...)', irrelevant and would take too much resources to clean the data. ***
    - 'Specific Location', irrelevant and would take too much resources to clean the data.
- Remove columns (2018):
    - 'Hectare', inconsistent with 2020 dataset.
    - 'Shift', inconsistent with 2020 dataset.
    - 'Hectare Squirrel Number', inconsistent with 2020 dataset.
    - 'Age', inconsistent with 2020 dataset.
    - 'Date', inconsistent with 2020 dataset, right??? We know 2018 data was collected in October [Autumn] and 2020 data was collected in March [Spring]
    - 'Combination of Primary and Highlight Color', inconsistent with 2020 dataset.
    - 'Color notes', inconsistent with 2020 dataset.
    - 'Location', irrelevant and would take too much resources to clean the data. ***
    - 'Above Ground Sighter Measurement', irrelevant and would take too much resources to clean the data. ***
    - 'Specific Location', inconsistent with 2020 dataset.
    - 'Lat/Long', delete due to double up.
##### 2020  /  2018 Column Names (in order... can be changed):
    - park_name                 /  - park_name [Central Park only - can create new column to match 2020 dataset column]
    - park_ID                   /  - park_ID [DOESN'T EXIST IN 2018 DATASET - or maybe we keep this as an ID might be easier to map?]
    - squirrel_ID               /  - 'Unique Squirrel ID' (change to squirrel_ID)
    - primary_fur_color         /  - primary_fur_color
    - highlights_in_fur_color   /  - 'Highlight Fur Color' (change to highlights_in_fur_color)
    - foraging
    - climbing
    - eating
    - running
    - chasing
    - shouting                  /  - 'Kuks' + 'Quaas' + 'Moans' (change to shouting)
    - sitting
    - digging
    - other_activities          /  - 'Other Activities' + 'Tail flags' + 'Tail twitches' (changed to other_activities)
    - interactions_with_humans  /  - 'Approaches' + 'Indifferent' + 'Runs from' + 'Other Interactions' (changed to interactions_with_humans) [MAYBE 'OTHER SPECIES'?]
    - other_observations        /  - other_observations [DOESN'T EXIST IN 2018 DATASET]
    - squirrel_latitude         /  - 'Y' (change to squirrel_latitude)
    - squirrel_longitude        /  - 'X' (change to squirrel_longitude)

- Rename columns: use underscores, all lower case.

In [5]:
# Drop columns
drop_columns = ['Area Name', 'Area ID', "Color Notes", "Location", "Above Ground (Height in Feet)", "Specific Location"]

reduced_2020 = s2020_nonull.drop(columns=drop_columns)
reduced_2020.head()

Park Name  Park ID Squirrel ID Primary Fur Color  \
0  Fort Tryon Park        1     A-01-01              Gray   
1  Fort Tryon Park        1     A-01-02              Gray   
2  Fort Tryon Park        1     A-01-03              Gray   
3  Fort Tryon Park        1     A-01-04              Gray   
4  Fort Tryon Park        1     A-01-05              Gray   

  Highlights in Fur Color                 Activities Interactions with Humans  \
0                   White                   Foraging              Indifferent   
1                   White                   Foraging              Indifferent   
2                   White  Eating, Digging something              Indifferent   
3                   White                    Running              Indifferent   
4                Cinnamon            Running, Eating              Indifferent   

  Other Notes or Observations  Squirrel Latitude (DD.DDDDDD)  \
0                         NaN                      40.859410   
1                Looks skinny                      40.859436   
2                         NaN                      40.859416   
3                         NaN                      40.859418   
4               She left food                      40.859493   

   Squirrel Longitude (-DD.DDDDDD)  
0                       -73.933936  
1                       -73.933937  
2                       -73.933894  
3                       -73.933895  
4                       -73.933590

In [6]:
# Rename the columns
cols_df = pd.DataFrame(reduced_2020.columns, columns=["name"])

# Define the regex pattern
pattern = '\((.*?)\)' # \( and \) to escape brackets, *.? matches any character (non-newline) zero or more times

new_columns = []
for row in cols_df["name"]:
    stripped_row = re.findall(pattern, row)

    # Remove the brackets and content
    if len(stripped_row) > 0:
        remove_substring = f' ({stripped_row[0].strip("[]")})'
        row = row.strip(remove_substring)

    # Use underscore and cast to lowercase
    new_columns.append(row.replace(" ", "_").lower())

# Update the columns
reduced_2020.columns = new_columns

reduced_2020.head()

park_name  park_id squirrel_id primary_fur_color  \
0  Fort Tryon Park        1     A-01-01              Gray   
1  Fort Tryon Park        1     A-01-02              Gray   
2  Fort Tryon Park        1     A-01-03              Gray   
3  Fort Tryon Park        1     A-01-04              Gray   
4  Fort Tryon Park        1     A-01-05              Gray   

  highlights_in_fur_color                 activities interactions_with_humans  \
0                   White                   Foraging              Indifferent   
1                   White                   Foraging              Indifferent   
2                   White  Eating, Digging something              Indifferent   
3                   White                    Running              Indifferent   
4                Cinnamon            Running, Eating              Indifferent   

  other_notes_or_observations  squirrel_latitude  squirrel_longitude  
0                         NaN          40.859410          -73.933936  
1                Looks skinny          40.859436          -73.933937  
2                         NaN          40.859416          -73.933894  
3                         NaN          40.859418          -73.933895  
4               She left food          40.859493          -73.933590

In [7]:
reduced_2020.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 192 entries, 0 to 432
Data columns (total 10 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   park_name                    192 non-null    object 
 1   park_id                      192 non-null    int64  
 2   squirrel_id                  192 non-null    object 
 3   primary_fur_color            192 non-null    object 
 4   highlights_in_fur_color      192 non-null    object 
 5   activities                   192 non-null    object 
 6   interactions_with_humans     192 non-null    object 
 7   other_notes_or_observations  96 non-null     object 
 8   squirrel_latitude            192 non-null    float64
 9   squirrel_longitude           192 non-null    float64
dtypes: float64(2), int64(1), object(7)
memory usage: 16.5+ KB


## Clean 'activities' column

In [8]:
# Create a DataFrame of the 'activities' column for cleaning
activities_df = reduced_2020[['squirrel_id', 'activities']].copy()
activities_df.head()

squirrel_id                 activities
0     A-01-01                   Foraging
1     A-01-02                   Foraging
2     A-01-03  Eating, Digging something
3     A-01-04                    Running
4     A-01-05            Running, Eating

In [9]:
row_list = []
for row in activities_df['activities']:
    row_list.append(row.split(", "))

activities_df['activity_list'] = row_list
activities_df.head()

squirrel_id                 activities                activity_list
0     A-01-01                   Foraging                   [Foraging]
1     A-01-02                   Foraging                   [Foraging]
2     A-01-03  Eating, Digging something  [Eating, Digging something]
3     A-01-04                    Running                    [Running]
4     A-01-05            Running, Eating            [Running, Eating]

In [10]:
idx_list = []
for idx, row in enumerate(activities_df['activity_list']):
    for word in row:
        if (word.startswith("Eating ")):
            print(idx, word)
            idx_list.append(idx)

129 Eating (or pretending to eat)
139 Eating (nuts)
178 Eating (bread crumbs)


In [11]:
activities_df.iloc[idx_list]

squirrel_id                              activities  \
330     C-18-01  Running, Eating (or pretending to eat)   
344     C-19-14                 Eating (nuts), Foraging   
415     D-22-27         Eating (bread crumbs), Foraging   

                                activity_list  
330  [Running, Eating (or pretending to eat)]  
344                 [Eating (nuts), Foraging]  
415         [Eating (bread crumbs), Foraging]

In [12]:
### CAN WE JUST DO THIS WITH THE reduced_2020 DATAFRAME TO AVOID HAVING TO MERGE? OR IS IT SAFER TO MAKE A COPY?
### I think it's easier to just make all the changes, because we'll create the boolean columns after this and merge it back?

# Combine 'child' activities that match 'parent' activities
# pattern = 'Eating \((.*?)\)'
# sitting = ["Hanging out", "Hanging", "Chilling", "Sitting in short tree", "Sticking out of a tree", "Very carefully watching a cat", "Watching #2", "Posing"]
# shouting = ["Vocalization at us", "Defending tree"]
# other_activities = ["Self-cleaning", "Sleeping", "Sleeping (Dead?)", "battery"]

pattern = 'eating \((.*?)\)'
sitting = ["hanging out", "hanging", "chilling", "sitting in short tree", "sticking out of a tree", "very carefully watching a cat", "watching #2", "posing"]
shouting = ["vocalization at us", "defending tree"]
other_activities = ["self-cleaning", "sleeping", "sleeping (dead?)", "battery"]

for row in activities_df['activity_list']:
    for idx, word in enumerate(row):
        row[idx] = word.lower()
        word = word.lower()

        # Check for "eating (...)"
        bracket = re.findall(pattern, word)
        if (len(bracket) > 0):
            row[idx] = "eating"

        # Check for "digging "
        if (word.startswith("digging ") or word == "burying"):
            row[idx] = "digging"

        if (word in shouting):
            row[idx] = "shouting"

        if (word == "nesting/gathering leaves"):
            row[idx] = "foraging"

        if (word in sitting):
            row[idx] = "sitting"

        if (word == "jumping"):
            row[idx] = "climbing"
        
        if (word in other_activities):
            row[idx] = "other_activities"

In [13]:
# Check the 'activities' distribution
flat_list = [value for row in activities_df['activity_list'] for value in row]
flat_df = pd.DataFrame(flat_list)
flat_df.value_counts()

foraging            72
climbing            52
eating              48
running             37
chasing             20
shouting            17
sitting             14
digging              5
other_activities     4
dtype: int64

In [14]:
### Create the boolean activity columns

# Create a list of the reduced activity names
column_names = list(set([word.lower() for word in flat_list]))

# Create new columns for each activity
for col_name in column_names:
    activities_df[col_name] = pd.Series(dtype=bool)

# Loop through the 'activity_list' and populate the boolean columns
for idx, row in activities_df.iterrows():
    for word in row['activity_list']:
        for activity in column_names:
            if (row[activity] == True):
                continue
            elif (word == activity):
                row[activity] = True
            else:
                row[activity] = False

# Display the DataFrame
activities_df.head()

squirrel_id                 activities      activity_list shouting chasing  \
0     A-01-01                   Foraging         [foraging]    False   False   
1     A-01-02                   Foraging         [foraging]    False   False   
2     A-01-03  Eating, Digging something  [eating, digging]    False   False   
3     A-01-04                    Running          [running]    False   False   
4     A-01-05            Running, Eating  [running, eating]    False   False   

  foraging sitting climbing eating other_activities digging running  
0     True   False    False  False            False   False   False  
1     True   False    False  False            False   False   False  
2    False   False    False   True            False    True   False  
3    False   False    False  False            False   False    True  
4    False   False    False   True            False   False    True

In [15]:
# Drop the 'activities' and 'activity_list' columns
activities_df = activities_df.drop(columns=['activities', 'activity_list'])
activities_df.head()

squirrel_id shouting chasing foraging sitting climbing eating  \
0     A-01-01    False   False     True   False    False  False   
1     A-01-02    False   False     True   False    False  False   
2     A-01-03    False   False    False   False    False   True   
3     A-01-04    False   False    False   False    False  False   
4     A-01-05    False   False    False   False    False   True   

  other_activities digging running  
0            False   False   False  
1            False   False   False  
2            False    True   False  
3            False   False    True  
4            False   False    True

In [16]:
# Merge with reduced_2020
s2020_updated = pd.merge(reduced_2020, activities_df, on="squirrel_id", how="inner")
s2020_updated.head()

park_name  park_id squirrel_id primary_fur_color  \
0  Fort Tryon Park        1     A-01-01              Gray   
1  Fort Tryon Park        1     A-01-02              Gray   
2  Fort Tryon Park        1     A-01-03              Gray   
3  Fort Tryon Park        1     A-01-04              Gray   
4  Fort Tryon Park        1     A-01-05              Gray   

  highlights_in_fur_color                 activities interactions_with_humans  \
0                   White                   Foraging              Indifferent   
1                   White                   Foraging              Indifferent   
2                   White  Eating, Digging something              Indifferent   
3                   White                    Running              Indifferent   
4                Cinnamon            Running, Eating              Indifferent   

  other_notes_or_observations  squirrel_latitude  squirrel_longitude shouting  \
0                         NaN          40.859410          -73.933936    False   
1                Looks skinny          40.859436          -73.933937    False   
2                         NaN          40.859416          -73.933894    False   
3                         NaN          40.859418          -73.933895    False   
4               She left food          40.859493          -73.933590    False   

  chasing foraging sitting climbing eating other_activities digging running  
0   False     True   False    False  False            False   False   False  
1   False     True   False    False  False            False   False   False  
2   False    False   False    False   True            False    True   False  
3   False    False   False    False  False            False   False    True  
4   False    False   False    False   True            False   False    True

In [17]:
# Drop the 'activities' column
s2020_updated = s2020_updated.drop(columns=['activities'])
s2020_updated.head()

park_name  park_id squirrel_id primary_fur_color  \
0  Fort Tryon Park        1     A-01-01              Gray   
1  Fort Tryon Park        1     A-01-02              Gray   
2  Fort Tryon Park        1     A-01-03              Gray   
3  Fort Tryon Park        1     A-01-04              Gray   
4  Fort Tryon Park        1     A-01-05              Gray   

  highlights_in_fur_color interactions_with_humans  \
0                   White              Indifferent   
1                   White              Indifferent   
2                   White              Indifferent   
3                   White              Indifferent   
4                Cinnamon              Indifferent   

  other_notes_or_observations  squirrel_latitude  squirrel_longitude shouting  \
0                         NaN          40.859410          -73.933936    False   
1                Looks skinny          40.859436          -73.933937    False   
2                         NaN          40.859416          -73.933894    False   
3                         NaN          40.859418          -73.933895    False   
4               She left food          40.859493          -73.933590    False   

  chasing foraging sitting climbing eating other_activities digging running  
0   False     True   False    False  False            False   False   False  
1   False     True   False    False  False            False   False   False  
2   False    False   False    False   True            False    True   False  
3   False    False   False    False  False            False   False    True  
4   False    False   False    False   True            False   False    True

## Clean 'interactions_with_humans' column

In [18]:
# Create a DataFrame of the 'activities' column for cleaning
interactions_df = s2020_updated[['squirrel_id', 'interactions_with_humans']].copy()
interactions_df.head()

squirrel_id interactions_with_humans
0     A-01-01              Indifferent
1     A-01-02              Indifferent
2     A-01-03              Indifferent
3     A-01-04              Indifferent
4     A-01-05              Indifferent

In [19]:
# Identify all the different interactions
print(interactions_df['interactions_with_humans'].value_counts())

# NOTE: 2018 equivalents = 'Approaches' + 'Indifferent' + 'Runs from' + 'Other Interactions'

Indifferent                                      127
Runs From                                         28
Approaches                                        20
Indifferent, Preoccupied by HAAWK                  6
Watches us from tree                               1
Runs From, watchful                                1
Watching us from tree - very interested in us      1
Approaches, watching us                            1
Runs From, watches us in short tree                1
Friendly                                           1
Indifferent, Runs From                             1
Approaches, Runs From                              1
Watching                                           1
Staring                                            1
Skittish to humans                                 1
Name: interactions_with_humans, dtype: int64


In [20]:
# Combine 'child' activities that match 'parent' activities
interact_options = ["approaches", "indifferent", "runs from"]

for idx, row in enumerate(interactions_df['interactions_with_humans']):
    # Convert to lowercase
    row = row.lower()
    
    # Account for rows with multiple interaction types
    multi_option = row.split(", ")
    if len(multi_option) > 1:
        count = 0
        for word in multi_option:
            # Increment if in interact_options
            if (word in interact_options):
                count += 1
            else:
                # Capture: "Indifferent, (not in interact_options)"
                if (row.startswith("indifferent, ")):
                    interactions_df.loc[idx, 'interactions_with_humans'] = ["indifferent"]

                # Captures the rows with "Watch%"
                else:
                    interactions_df.loc[idx, 'interactions_with_humans'] = [(multi_option[0]), "watching"]

            # All options are in interact_options
            if (count == len(multi_option)):
                interactions_df.loc[idx, 'interactions_with_humans'] = multi_option # [word.replace(" ", "_") for word in multi_option]

    # Account for rows with a single interaction
    else:
        # If applicable, filter child to parent
        if (row.startswith("watch") or row == "staring"):
            interactions_df.loc[idx, 'interactions_with_humans'] = ["watching"]
        if (row == "skittish to humans"):
            interactions_df.loc[idx, 'interactions_with_humans'] = ["runs from"]
        if (row == "friendly"):
            interactions_df.loc[idx, 'interactions_with_humans'] = ["approaches"]
        if (row in interact_options):
            # Convert the row to a list, for simplicity later
            interactions_df.loc[idx, 'interactions_with_humans'] = [row] #[row.replace(" ", "_")]

In [21]:
# Check the updated distribution
print(interactions_df['interactions_with_humans'].value_counts())

[indifferent]               133
[runs from]                  29
[approaches]                 21
[watching]                    4
[runs from, watching]         2
[approaches, watching]        1
[indifferent, runs from]      1
[approaches, runs from]       1
Name: interactions_with_humans, dtype: int64


In [22]:
### Create the boolean interaction columns
unique_interactions = list(set([value for row in interactions_df['interactions_with_humans'] for value in row]))

# Create new columns for each interaction
for col_name in unique_interactions:
    interactions_df[col_name] = pd.Series(dtype=bool)

# Loop through the 'activity_list' and populate the boolean columns
for idx, row in interactions_df.iterrows():
    for word in row['interactions_with_humans']:
        for interaction in unique_interactions:
            if (row[interaction] == True):
                continue
            elif (word == interaction):
                row[interaction] = True
            else:
                row[interaction] = False

# Display the DataFrame
interactions_df.head()

squirrel_id interactions_with_humans approaches watching indifferent  \
0     A-01-01            [indifferent]      False    False        True   
1     A-01-02            [indifferent]      False    False        True   
2     A-01-03            [indifferent]      False    False        True   
3     A-01-04            [indifferent]      False    False        True   
4     A-01-05            [indifferent]      False    False        True   

  runs from  
0     False  
1     False  
2     False  
3     False  
4     False

In [23]:
# Drop the 'interactions_with_humans' column
interactions_df = interactions_df.drop(columns=['interactions_with_humans'])
interactions_df.head()

squirrel_id approaches watching indifferent runs from
0     A-01-01      False    False        True     False
1     A-01-02      False    False        True     False
2     A-01-03      False    False        True     False
3     A-01-04      False    False        True     False
4     A-01-05      False    False        True     False

In [24]:
# Merge with s2020_updated
s2020_booleans = pd.merge(s2020_updated, interactions_df, on="squirrel_id", how="inner")
s2020_booleans.head()

park_name  park_id squirrel_id primary_fur_color  \
0  Fort Tryon Park        1     A-01-01              Gray   
1  Fort Tryon Park        1     A-01-02              Gray   
2  Fort Tryon Park        1     A-01-03              Gray   
3  Fort Tryon Park        1     A-01-04              Gray   
4  Fort Tryon Park        1     A-01-05              Gray   

  highlights_in_fur_color interactions_with_humans  \
0                   White              Indifferent   
1                   White              Indifferent   
2                   White              Indifferent   
3                   White              Indifferent   
4                Cinnamon              Indifferent   

  other_notes_or_observations  squirrel_latitude  squirrel_longitude shouting  \
0                         NaN          40.859410          -73.933936    False   
1                Looks skinny          40.859436          -73.933937    False   
2                         NaN          40.859416          -73.933894    False   
3                         NaN          40.859418          -73.933895    False   
4               She left food          40.859493          -73.933590    False   

   ... sitting climbing eating other_activities digging running approaches  \
0  ...   False    False  False            False   False   False      False   
1  ...   False    False  False            False   False   False      False   
2  ...   False    False   True            False    True   False      False   
3  ...   False    False  False            False   False    True      False   
4  ...   False    False   True            False   False    True      False   

  watching indifferent runs from  
0    False        True     False  
1    False        True     False  
2    False        True     False  
3    False        True     False  
4    False        True     False  

[5 rows x 22 columns]

In [25]:
# Drop the 'interactions_with_humans' column
s2020_booleans = s2020_booleans.drop(columns=['interactions_with_humans'])
s2020_booleans.head()

park_name  park_id squirrel_id primary_fur_color  \
0  Fort Tryon Park        1     A-01-01              Gray   
1  Fort Tryon Park        1     A-01-02              Gray   
2  Fort Tryon Park        1     A-01-03              Gray   
3  Fort Tryon Park        1     A-01-04              Gray   
4  Fort Tryon Park        1     A-01-05              Gray   

  highlights_in_fur_color other_notes_or_observations  squirrel_latitude  \
0                   White                         NaN          40.859410   
1                   White                Looks skinny          40.859436   
2                   White                         NaN          40.859416   
3                   White                         NaN          40.859418   
4                Cinnamon               She left food          40.859493   

   squirrel_longitude shouting chasing  ... sitting climbing eating  \
0          -73.933936    False   False  ...   False    False  False   
1          -73.933937    False   False  ...   False    False  False   
2          -73.933894    False   False  ...   False    False   True   
3          -73.933895    False   False  ...   False    False  False   
4          -73.933590    False   False  ...   False    False   True   

  other_activities digging running approaches watching indifferent runs from  
0            False   False   False      False    False        True     False  
1            False   False   False      False    False        True     False  
2            False    True   False      False    False        True     False  
3            False   False    True      False    False        True     False  
4            False   False    True      False    False        True     False  

[5 rows x 21 columns]

## Confirm Data Types

In [26]:
# Confirm boolean columns are of "bool" type
s2020_booleans.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 192 entries, 0 to 191
Data columns (total 21 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   park_name                    192 non-null    object 
 1   park_id                      192 non-null    int64  
 2   squirrel_id                  192 non-null    object 
 3   primary_fur_color            192 non-null    object 
 4   highlights_in_fur_color      192 non-null    object 
 5   other_notes_or_observations  96 non-null     object 
 6   squirrel_latitude            192 non-null    float64
 7   squirrel_longitude           192 non-null    float64
 8   shouting                     192 non-null    object 
 9   chasing                      192 non-null    object 
 10  foraging                     192 non-null    object 
 11  sitting                      192 non-null    object 
 12  climbing                     192 non-null    object 
 13  eating              

In [27]:
# Convert boolean columns to "bool" type
bool_columns = s2020_booleans.columns[8:]
s2020_booleans[bool_columns] = s2020_booleans[bool_columns].astype(bool)

In [28]:
s2020_booleans.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 192 entries, 0 to 191
Data columns (total 21 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   park_name                    192 non-null    object 
 1   park_id                      192 non-null    int64  
 2   squirrel_id                  192 non-null    object 
 3   primary_fur_color            192 non-null    object 
 4   highlights_in_fur_color      192 non-null    object 
 5   other_notes_or_observations  96 non-null     object 
 6   squirrel_latitude            192 non-null    float64
 7   squirrel_longitude           192 non-null    float64
 8   shouting                     192 non-null    bool   
 9   chasing                      192 non-null    bool   
 10  foraging                     192 non-null    bool   
 11  sitting                      192 non-null    bool   
 12  climbing                     192 non-null    bool   
 13  eating              

# 2018 Dataset - Squirrels

In [29]:
# Create dataframe from CSV
s2018_df = pd.read_csv(squirrel_2018)

# Display DataFrame and its shape
print(f"2018 Dataset: {s2018_df.shape}")
s2018_df.head()

2018 Dataset: (3023, 31)


X          Y Unique Squirrel ID Hectare Shift      Date  \
0 -73.956134  40.794082     37F-PM-1014-03     37F    PM  10142018   
1 -73.968857  40.783783     21B-AM-1019-04     21B    AM  10192018   
2 -73.974281  40.775534     11B-PM-1014-08     11B    PM  10142018   
3 -73.959641  40.790313     32E-PM-1017-14     32E    PM  10172018   
4 -73.970268  40.776213     13E-AM-1017-05     13E    AM  10172018   

   Hectare Squirrel Number    Age Primary Fur Color Highlight Fur Color  ...  \
0                        3    NaN               NaN                 NaN  ...   
1                        4    NaN               NaN                 NaN  ...   
2                        8    NaN              Gray                 NaN  ...   
3                       14  Adult              Gray                 NaN  ...   
4                        5  Adult              Gray            Cinnamon  ...   

    Kuks  Quaas  Moans Tail flags Tail twitches  Approaches  Indifferent  \
0  False  False  False      False         False       False        False   
1  False  False  False      False         False       False        False   
2  False  False  False      False         False       False        False   
3  False  False  False      False         False       False        False   
4  False  False  False      False         False       False        False   

   Runs from  Other Interactions                                    Lat/Long  
0      False                 NaN  POINT (-73.9561344937861 40.7940823884086)  
1      False                 NaN  POINT (-73.9688574691102 40.7837825208444)  
2      False                 NaN  POINT (-73.97428114848522 40.775533619083)  
3       True                 NaN  POINT (-73.9596413903948 40.7903128889029)  
4      False                 NaN  POINT (-73.9702676472613 40.7762126854894)  

[5 rows x 31 columns]

In [30]:
# Identify the non-null count and data types
s2018_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3023 entries, 0 to 3022
Data columns (total 31 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   X                                           3023 non-null   float64
 1   Y                                           3023 non-null   float64
 2   Unique Squirrel ID                          3023 non-null   object 
 3   Hectare                                     3023 non-null   object 
 4   Shift                                       3023 non-null   object 
 5   Date                                        3023 non-null   int64  
 6   Hectare Squirrel Number                     3023 non-null   int64  
 7   Age                                         2902 non-null   object 
 8   Primary Fur Color                           2968 non-null   object 
 9   Highlight Fur Color                         1937 non-null   object 
 10  Combination 

In [31]:
# 2018 Dataset

# Drop null values for minimum requirement columns
s2018_nonull = s2018_df.dropna(
    subset=[
        "Primary Fur Color",
        "Highlight Fur Color",
        "Running",
        "Chasing",
        "Climbing",
        "Eating",
        "Foraging",
        "Approaches",
        "Indifferent",
        "Runs from",
        "X",
        "Y"],
    how="any")

# NOTE: If you add "Other Interactions", dataset is reduced to 170 rows
# Number of unique values for "Other Notes or Observations"
print(f'Unique "other interactions": {s2018_nonull["Other Interactions"].nunique()}') # 152 unique interactions

# Display DataFrame and its shape
print(f"{s2018_nonull.shape}")
s2018_nonull.head()

Unique "other interactions": 152
(1937, 31)


X          Y Unique Squirrel ID Hectare Shift      Date  \
4  -73.970268  40.776213     13E-AM-1017-05     13E    AM  10172018   
5  -73.968361  40.772591     11H-AM-1010-03     11H    AM  10102018   
9  -73.972250  40.774288     11D-AM-1010-03     11D    AM  10102018   
10 -73.969506  40.782351     20B-PM-1013-05     20B    PM  10132018   
12 -73.953217  40.791967     36I-PM-1007-01     36I    PM  10072018   

    Hectare Squirrel Number    Age Primary Fur Color Highlight Fur Color  ...  \
4                         5  Adult              Gray            Cinnamon  ...   
5                         3  Adult          Cinnamon               White  ...   
9                         3  Adult              Gray            Cinnamon  ...   
10                        5  Adult              Gray               White  ...   
12                        1  Adult              Gray            Cinnamon  ...   

     Kuks  Quaas  Moans Tail flags Tail twitches  Approaches  Indifferent  \
4   False  False  False      False         False       False        False   
5   False  False  False      False          True       False         True   
9   False  False  False      False         False       False         True   
10  False  False  False      False         False       False         True   
12  False  False  False      False         False       False         True   

    Runs from  Other Interactions                                    Lat/Long  
4       False                 NaN  POINT (-73.9702676472613 40.7762126854894)  
5       False                 NaN  POINT (-73.9683613516225 40.7725908847499)  
9       False                 NaN  POINT (-73.9722500196844 40.7742879599026)  
10      False                 NaN  POINT (-73.9695063535333 40.7823507678183)  
12      False                 NaN  POINT (-73.9532170504865 40.7919669739962)  

[5 rows x 31 columns]

In [32]:
# Rename the columns
cols_df = pd.DataFrame(s2018_nonull.columns, columns=["name"])

# Define the regex pattern
pattern = '\((.*?)\)' # \( and \) to escape brackets, *.? matches any character (non-newline) zero or more times

new_columns = []
for row in cols_df["name"]:
    # Use underscore and cast to lowercase
    new_columns.append(row.replace(" ", "_").lower())

# Update the columns
s2018_nonull.columns = new_columns

s2018_nonull.head()

x          y unique_squirrel_id hectare shift      date  \
4  -73.970268  40.776213     13E-AM-1017-05     13E    AM  10172018   
5  -73.968361  40.772591     11H-AM-1010-03     11H    AM  10102018   
9  -73.972250  40.774288     11D-AM-1010-03     11D    AM  10102018   
10 -73.969506  40.782351     20B-PM-1013-05     20B    PM  10132018   
12 -73.953217  40.791967     36I-PM-1007-01     36I    PM  10072018   

    hectare_squirrel_number    age primary_fur_color highlight_fur_color  ...  \
4                         5  Adult              Gray            Cinnamon  ...   
5                         3  Adult          Cinnamon               White  ...   
9                         3  Adult              Gray            Cinnamon  ...   
10                        5  Adult              Gray               White  ...   
12                        1  Adult              Gray            Cinnamon  ...   

     kuks  quaas  moans tail_flags tail_twitches  approaches  indifferent  \
4   False  False  False      False         False       False        False   
5   False  False  False      False          True       False         True   
9   False  False  False      False         False       False         True   
10  False  False  False      False         False       False         True   
12  False  False  False      False         False       False         True   

    runs_from  other_interactions                                    lat/long  
4       False                 NaN  POINT (-73.9702676472613 40.7762126854894)  
5       False                 NaN  POINT (-73.9683613516225 40.7725908847499)  
9       False                 NaN  POINT (-73.9722500196844 40.7742879599026)  
10      False                 NaN  POINT (-73.9695063535333 40.7823507678183)  
12      False                 NaN  POINT (-73.9532170504865 40.7919669739962)  

[5 rows x 31 columns]

In [33]:
s2018_nonull.columns

Index(['x', 'y', 'unique_squirrel_id', 'hectare', 'shift', 'date',
       'hectare_squirrel_number', 'age', 'primary_fur_color',
       'highlight_fur_color', 'combination_of_primary_and_highlight_color',
       'color_notes', 'location', 'above_ground_sighter_measurement',
       'specific_location', 'running', 'chasing', 'climbing', 'eating',
       'foraging', 'other_activities', 'kuks', 'quaas', 'moans', 'tail_flags',
       'tail_twitches', 'approaches', 'indifferent', 'runs_from',
       'other_interactions', 'lat/long'],
      dtype='object')

In [34]:
## Delete unwanted columns but maintain original copy
# s2018_drop_columns = s2018_nonull.drop(columns=['hectare', 'shift', 'hectare_squirrel_number', 'age', 'other_interactions'], inplace=False)
s2018_drop_columns = s2018_nonull.drop(columns=[
    'hectare',
    'shift',
    'hectare_squirrel_number',
    'combination_of_primary_and_highlight_color',
    'color_notes',
    'location',
    'above_ground_sighter_measurement',
    'specific_location',
    'tail_flags',
    'tail_twitches'
]).reset_index(drop=True)

# Rename columns to match 2020 dataset
s2018_rename_columns = s2018_drop_columns.rename(columns={
    "x": "squirrel_longitude",
    "y": "squirrel_latitude",
    "highlight_fur_colour": "highlights_in_fur_color",
    "unique_squirrel_id": "squirrel_id"
})

s2018_rename_columns.head()

squirrel_longitude  squirrel_latitude     squirrel_id      date    age  \
0          -73.970268          40.776213  13E-AM-1017-05  10172018  Adult   
1          -73.968361          40.772591  11H-AM-1010-03  10102018  Adult   
2          -73.972250          40.774288  11D-AM-1010-03  10102018  Adult   
3          -73.969506          40.782351  20B-PM-1013-05  10132018  Adult   
4          -73.953217          40.791967  36I-PM-1007-01  10072018  Adult   

  primary_fur_color highlight_fur_color  running  chasing  climbing  ...  \
0              Gray            Cinnamon    False    False     False  ...   
1          Cinnamon               White    False    False     False  ...   
2              Gray            Cinnamon    False    False      True  ...   
3              Gray               White    False    False     False  ...   
4              Gray            Cinnamon    False    False     False  ...   

   foraging  other_activities   kuks  quaas  moans  approaches  indifferent  \
0      True               NaN  False  False  False       False        False   
1      True               NaN  False  False  False       False         True   
2     False          grooming  False  False  False       False         True   
3      True               NaN  False  False  False       False         True   
4      True               NaN  False  False  False       False         True   

   runs_from  other_interactions                                    lat/long  
0      False                 NaN  POINT (-73.9702676472613 40.7762126854894)  
1      False                 NaN  POINT (-73.9683613516225 40.7725908847499)  
2      False                 NaN  POINT (-73.9722500196844 40.7742879599026)  
3      False                 NaN  POINT (-73.9695063535333 40.7823507678183)  
4      False                 NaN  POINT (-73.9532170504865 40.7919669739962)  

[5 rows x 21 columns]

## Combine 'kuks', 'quaas', 'moans' to 'shouting'

In [35]:
# Isolate the "shouting" equivalent columns - kuks, quaas, moans
shouting_df = s2018_rename_columns[['squirrel_id', 'kuks', 'quaas', 'moans']].copy()

shouting_df['shouting'] = pd.Series(dtype=bool)
shouting_df

squirrel_id   kuks  quaas  moans shouting
0     13E-AM-1017-05  False  False  False      NaN
1     11H-AM-1010-03  False  False  False      NaN
2     11D-AM-1010-03  False  False  False      NaN
3     20B-PM-1013-05  False  False  False      NaN
4     36I-PM-1007-01  False  False  False      NaN
...              ...    ...    ...    ...      ...
1932  21F-PM-1018-02  False  False  False      NaN
1933  19A-PM-1013-05  False  False  False      NaN
1934  22D-PM-1012-07  False  False  False      NaN
1935  29B-PM-1010-02  False  False  False      NaN
1936   5E-PM-1012-01  False  False  False      NaN

[1937 rows x 5 columns]

In [36]:
# If any of the shouting_cols are True, set 'shouting' to True
shouting_cols = shouting_df.columns[1:4]

for idx, row in shouting_df.iterrows():
    for col in shouting_cols:
        if row[col] == True:
            shouting_df.loc[idx, 'shouting'] = True
            break
        else:
            shouting_df.loc[idx, 'shouting'] = False

shouting_df['shouting'].value_counts()

False    1859
True       78
Name: shouting, dtype: int64

In [37]:
# Drop the shouting_cols
shouting_df = shouting_df.drop(columns=shouting_cols)
shouting_df.head()

squirrel_id shouting
0  13E-AM-1017-05    False
1  11H-AM-1010-03    False
2  11D-AM-1010-03    False
3  20B-PM-1013-05    False
4  36I-PM-1007-01    False

In [38]:
# Merge with s2018_rename_columns
s2018_booleans = pd.merge(s2018_rename_columns, shouting_df, on="squirrel_id", how="inner")
s2018_booleans.head()

squirrel_longitude  squirrel_latitude     squirrel_id      date    age  \
0          -73.970268          40.776213  13E-AM-1017-05  10172018  Adult   
1          -73.968361          40.772591  11H-AM-1010-03  10102018  Adult   
2          -73.972250          40.774288  11D-AM-1010-03  10102018  Adult   
3          -73.969506          40.782351  20B-PM-1013-05  10132018  Adult   
4          -73.953217          40.791967  36I-PM-1007-01  10072018  Adult   

  primary_fur_color highlight_fur_color  running  chasing  climbing  ...  \
0              Gray            Cinnamon    False    False     False  ...   
1          Cinnamon               White    False    False     False  ...   
2              Gray            Cinnamon    False    False      True  ...   
3              Gray               White    False    False     False  ...   
4              Gray            Cinnamon    False    False     False  ...   

   other_activities   kuks  quaas  moans  approaches  indifferent  runs_from  \
0               NaN  False  False  False       False        False      False   
1               NaN  False  False  False       False         True      False   
2          grooming  False  False  False       False         True      False   
3               NaN  False  False  False       False         True      False   
4               NaN  False  False  False       False         True      False   

   other_interactions                                    lat/long shouting  
0                 NaN  POINT (-73.9702676472613 40.7762126854894)    False  
1                 NaN  POINT (-73.9683613516225 40.7725908847499)    False  
2                 NaN  POINT (-73.9722500196844 40.7742879599026)    False  
3                 NaN  POINT (-73.9695063535333 40.7823507678183)    False  
4                 NaN  POINT (-73.9532170504865 40.7919669739962)    False  

[5 rows x 22 columns]

In [39]:
# Drop the shouting_cols from s2018_booleans
s2018_booleans = s2018_booleans.drop(columns=shouting_cols)
s2018_booleans.head()

squirrel_longitude  squirrel_latitude     squirrel_id      date    age  \
0          -73.970268          40.776213  13E-AM-1017-05  10172018  Adult   
1          -73.968361          40.772591  11H-AM-1010-03  10102018  Adult   
2          -73.972250          40.774288  11D-AM-1010-03  10102018  Adult   
3          -73.969506          40.782351  20B-PM-1013-05  10132018  Adult   
4          -73.953217          40.791967  36I-PM-1007-01  10072018  Adult   

  primary_fur_color highlight_fur_color  running  chasing  climbing  eating  \
0              Gray            Cinnamon    False    False     False   False   
1          Cinnamon               White    False    False     False   False   
2              Gray            Cinnamon    False    False      True   False   
3              Gray               White    False    False     False   False   
4              Gray            Cinnamon    False    False     False   False   

   foraging other_activities  approaches  indifferent  runs_from  \
0      True              NaN       False        False      False   
1      True              NaN       False         True      False   
2     False         grooming       False         True      False   
3      True              NaN       False         True      False   
4      True              NaN       False         True      False   

  other_interactions                                    lat/long shouting  
0                NaN  POINT (-73.9702676472613 40.7762126854894)    False  
1                NaN  POINT (-73.9683613516225 40.7725908847499)    False  
2                NaN  POINT (-73.9722500196844 40.7742879599026)    False  
3                NaN  POINT (-73.9695063535333 40.7823507678183)    False  
4                NaN  POINT (-73.9532170504865 40.7919669739962)    False

## Change Data Types

In [40]:
from datetime import datetime
s2018_booleans['date'] = s2018_booleans['date'].apply(lambda x: datetime.strptime(str(x), '%m%d%Y').strftime('%Y-%m-%d'))

In [41]:
s2018_booleans.head()

squirrel_longitude  squirrel_latitude     squirrel_id        date    age  \
0          -73.970268          40.776213  13E-AM-1017-05  2018-10-17  Adult   
1          -73.968361          40.772591  11H-AM-1010-03  2018-10-10  Adult   
2          -73.972250          40.774288  11D-AM-1010-03  2018-10-10  Adult   
3          -73.969506          40.782351  20B-PM-1013-05  2018-10-13  Adult   
4          -73.953217          40.791967  36I-PM-1007-01  2018-10-07  Adult   

  primary_fur_color highlight_fur_color  running  chasing  climbing  eating  \
0              Gray            Cinnamon    False    False     False   False   
1          Cinnamon               White    False    False     False   False   
2              Gray            Cinnamon    False    False      True   False   
3              Gray               White    False    False     False   False   
4              Gray            Cinnamon    False    False     False   False   

   foraging other_activities  approaches  indifferent  runs_from  \
0      True              NaN       False        False      False   
1      True              NaN       False         True      False   
2     False         grooming       False         True      False   
3      True              NaN       False         True      False   
4      True              NaN       False         True      False   

  other_interactions                                    lat/long shouting  
0                NaN  POINT (-73.9702676472613 40.7762126854894)    False  
1                NaN  POINT (-73.9683613516225 40.7725908847499)    False  
2                NaN  POINT (-73.9722500196844 40.7742879599026)    False  
3                NaN  POINT (-73.9695063535333 40.7823507678183)    False  
4                NaN  POINT (-73.9532170504865 40.7919669739962)    False

In [42]:
s2018_booleans.info()
# The 'date' column is still needs to be converted to the correct datatime type.

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1945 entries, 0 to 1944
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   squirrel_longitude   1945 non-null   float64
 1   squirrel_latitude    1945 non-null   float64
 2   squirrel_id          1945 non-null   object 
 3   date                 1945 non-null   object 
 4   age                  1882 non-null   object 
 5   primary_fur_color    1945 non-null   object 
 6   highlight_fur_color  1945 non-null   object 
 7   running              1945 non-null   bool   
 8   chasing              1945 non-null   bool   
 9   climbing             1945 non-null   bool   
 10  eating               1945 non-null   bool   
 11  foraging             1945 non-null   bool   
 12  other_activities     285 non-null    object 
 13  approaches           1945 non-null   bool   
 14  indifferent          1945 non-null   bool   
 15  runs_from            1945 non-null   b

## Investigate 'other_activities' column

In [43]:
# Check the "other activities" column
other_activities = pd.DataFrame(s2018_booleans['other_activities'].value_counts())
other_activities

other_activities
digging                                           14
sitting                                            9
playing                                            8
burying                                            6
nut in mouth                                       4
...                                              ...
jumping tree to tree                               1
carrying nut away                                  1
chasing #2 and climbing tree w/ #2                 1
standing upright                                   1
eating (on branch)                                 1

[219 rows x 1 columns]

In [44]:
# Isolate the "other activities" column
other_df = s2018_booleans[['squirrel_id', 'other_activities']].copy()

# Only include rows with a non-null value - REGEX DOESN'T WORK IF array has NaN values!
other_df = other_df.dropna(how="any")

# Equivalent columns from 2020 currently missing from 2018
missing_cols = ['sitting', 'digging']

# Create new columns for each activity
for col_name in missing_cols:
    other_df[col_name] = pd.Series(dtype=bool)

In [45]:
# CONVERT THIS TO A DICTIONARY LATER, for simplicity!
# Combine activities
patterns = [
    'sit', 'watch', 'staring', 'posing', 'hanging out', 'chillin', 'dig', 'bury', 'chas', 'climb', 'eat', 'hop', 'foraging', 'running']
activity_col = [
    'sitting', 'sitting', 'sitting', 'sitting', 'sitting', 'sitting', 'digging', 'digging', 'chasing', 'climbing', 'eating', 'running', 'foraging', 'running']

all_indices = []
for idx_pattern, pattern in enumerate(patterns):
    idx_sit = list(other_df[other_df['other_activities'].str.contains(pattern)].index)
    all_indices.append(idx_sit)
    
    for idx in idx_sit:
        other_df.loc[idx, activity_col[idx_pattern]] = True
# idx_sit = list(other_df[other_df['other_activities'].str.contains(pattern_sitting)].index)

# for idx in idx_sit:
#     other_df.loc[idx, 'sitting'] = True

other_df[other_df['sitting'] == True]
# for idx, row in other_df.iterrows():
#     print(row['other_activities'])

squirrel_id                                   other_activities  \
11    12I-AM-1013-01                                            sitting   
71    16G-AM-1010-03            was hanging out w/ #2 until i showed up   
111   19B-AM-1019-02                                             posing   
113    4G-AM-1006-02                                    sitting in tree   
146    4E-AM-1020-01                                 sitting very still   
178   22C-AM-1013-01                       sitting still,tail over back   
188    4D-PM-1008-08         climbed fence staring at me - went up tree   
257   22G-PM-1018-03                                           chilling   
304   42D-PM-1013-01                                watching passers-by   
308   14H-AM-1007-02                                            sitting   
382   14B-PM-1014-06                                        watching me   
430   34E-PM-1006-01                     sitting and watching on a tree   
486   12E-AM-1017-08                                           chilling   
705   35H-AM-1013-02                                            sitting   
712   21H-PM-1018-01                                            staring   
733   15H-PM-1008-01                                        watching me   
752   14F-PM-1012-05                                            sitting   
757   16E-PM-1018-05                                            staring   
803    6G-AM-1008-01                                            sitting   
844   30I-PM-1006-01                                      sitting there   
882    7F-AM-1008-14                                            sitting   
894   35E-AM-1020-04  hunkered down,tail over body; sitting perfectl...   
922   35C-AM-1007-01                                      sitting still   
954   25A-AM-1013-01                                      staring at me   
1030  36A-PM-1013-04                                            sitting   
1072  38E-AM-1010-10                                           chillin'   
1254  33A-PM-1018-02                         hanging out,watching world   
1288  22H-PM-1007-02                             watching me scratching   
1340   2I-PM-1006-01                                      staring at me   
1342  37B-PM-1013-03                                           watching   
1356  19G-PM-1017-01                                           watching   
1370  35B-PM-1013-01  did not notice dog sitting a few yards away (1...   
1418  20F-PM-1007-07                                           watching   
1528  23B-PM-1012-04                          watching me - didn't move   
1577  17F-AM-1007-04                                           chilling   
1601  10C-AM-1018-02                                 sitting in the sun   
1607  10F-PM-1019-01                                            sitting   
1625  33E-AM-1014-13                                            sitting   
1638   4E-AM-1020-04                                           chillin'   
1689  14E-PM-1017-02                                            staring   
1908  18I-PM-1007-04                                 walking sitting up   

     sitting digging chasing climbing eating running foraging  
11      True     NaN     NaN      NaN    NaN     NaN      NaN  
71      True     NaN     NaN      NaN    NaN     NaN      NaN  
111     True     NaN     NaN      NaN    NaN     NaN      NaN  
113     True     NaN     NaN      NaN    NaN     NaN      NaN  
146     True     NaN     NaN      NaN    NaN     NaN      NaN  
178     True     NaN     NaN      NaN    NaN     NaN      NaN  
188     True     NaN     NaN     True    NaN     NaN      NaN  
257     True     NaN     NaN      NaN    NaN     NaN      NaN  
304     True     NaN     NaN      NaN    NaN     NaN      NaN  
308     True     NaN     NaN      NaN    NaN     NaN      NaN  
382     True     NaN     NaN      NaN    NaN     NaN      NaN  
430     True     NaN     NaN      NaN    NaN     NaN      NaN  
486     True     NaN     N

In [46]:
flat_indices = [value for row in all_indices for value in row]
len(flat_indices)

172

In [47]:
len(set(flat_indices))

163

In [48]:
reduced_other = other_df.drop(index=list(set(flat_indices)))
print(len(reduced_other))
reduced_other.tail(60) # Leave these out for now?
# Just get a clean dataset and add these in later if required?

122


squirrel_id                                   other_activities  \
953   37C-AM-1012-02                                            caching   
992   39I-PM-1013-04                                            playing   
1005  33E-PM-1012-04                                            playing   
1010   1D-AM-1006-06                                           tumbling   
1046  18I-PM-1007-01                                hanging upside down   
1057   7H-AM-1006-01                                       nut in mouth   
1107  32F-AM-1012-02                                   curious about me   
1115  38E-PM-1014-05                                             statue   
1135   3D-PM-1010-14                                         crying out   
1151   7I-PM-1013-07                        playing,interacting with #6   
1159  33E-PM-1012-06                                            playing   
1191   6C-AM-1007-07                                            ambling   
1239   5F-PM-1007-06                                   jumping on fence   
1241  30B-PM-1019-05                                       hiding a nut   
1278  29B-PM-1010-01                                    scratching self   
1297  21F-AM-1018-03                                               dead   
1303  34I-AM-1012-02                                             hiding   
1304  11B-PM-1014-06                                           cleaning   
1317   3G-PM-1013-02                wandering,interacting with tourists   
1327  36F-AM-1017-05  staying very still to avoid two very curious d...   
1329  38I-PM-1014-02                          scampering w/ squirrel #1   
1358  38A-AM-1019-03                                 stood on back legs   
1371  19G-PM-1017-02                                     patting ground   
1373  33D-PM-1006-01                                   hanging on fence   
1374   9B-AM-1010-06                                           cleaning   
1390  38F-AM-1018-05                                        ran up tree   
1393  37G-PM-1007-05                                    jumping around!   
1422  18I-PM-1007-02                                hanging upside down   
1480  18H-AM-1018-01                                           playing?   
1483  42A-PM-1014-01                     up on hind legs looking around   
1485   9B-AM-1010-09                                      food in mouth   
1494  11D-AM-1010-02                                            walking   
1507  42C-AM-1007-01                                       nut in mouth   
1511   2C-PM-1013-06                               cleaning face & tail   
1519   8F-PM-1012-07                                          gathering   
1522  37C-AM-1012-01                                            caching   
1525  32H-AM-1010-01                not sure; walking-stop-walking-stop   
1546   3B-AM-1010-02                                     together w/ #3   
1566  42F-PM-1017-01                                            playing   
1579  34I-AM-1012-03                                       nut in mouth   
1580  15B-AM-1014-02                                grooming,scratching   
1587   1A-AM-1007-03                                    looking in bush   
1611   5C-AM-1006-04                                           relaxing   
1616  15G-PM-1019-06                                        scratching!   
1632  12F-AM-1020-01                                            jumping   
1686  15F-AM-1007-06                                       pair with #5   
1715   3H-AM-1008-01                                teasing barking dog   
1720  38A-AM-1019-05                              standing on back legs   
1729  15D-PM-1017-06                                            walking   
1740  34A-PM-1013-01                                    took a chestnut   
1774   7G-PM-1018-06                                           grooming   
1819  32I-PM-1007-01                                             chirps   
1851  33H-AM-1019-01              

In [49]:
# Merge other_df back to 2018 dataset
print(len(other_df))
other_df.head()

285


squirrel_id       other_activities sitting digging chasing climbing  \
2   11D-AM-1010-03               grooming     NaN     NaN     NaN      NaN   
11  12I-AM-1013-01                sitting    True     NaN     NaN      NaN   
19  33H-AM-1019-02  wrestling with mother     NaN     NaN     NaN      NaN   
25   2B-PM-1013-01     running (with nut)     NaN     NaN     NaN      NaN   
29   6I-PM-1013-06        playing with #5     NaN     NaN     NaN      NaN   

   eating running foraging  
2     NaN     NaN      NaN  
11    NaN     NaN      NaN  
19    NaN     NaN      NaN  
25    NaN    True      NaN  
29    NaN     NaN      NaN

In [50]:
print(len(s2018_booleans))
s2018_booleans.head()

1945


squirrel_longitude  squirrel_latitude     squirrel_id        date    age  \
0          -73.970268          40.776213  13E-AM-1017-05  2018-10-17  Adult   
1          -73.968361          40.772591  11H-AM-1010-03  2018-10-10  Adult   
2          -73.972250          40.774288  11D-AM-1010-03  2018-10-10  Adult   
3          -73.969506          40.782351  20B-PM-1013-05  2018-10-13  Adult   
4          -73.953217          40.791967  36I-PM-1007-01  2018-10-07  Adult   

  primary_fur_color highlight_fur_color  running  chasing  climbing  eating  \
0              Gray            Cinnamon    False    False     False   False   
1          Cinnamon               White    False    False     False   False   
2              Gray            Cinnamon    False    False      True   False   
3              Gray               White    False    False     False   False   
4              Gray            Cinnamon    False    False     False   False   

   foraging other_activities  approaches  indifferent  runs_from  \
0      True              NaN       False        False      False   
1      True              NaN       False         True      False   
2     False         grooming       False         True      False   
3      True              NaN       False         True      False   
4      True              NaN       False         True      False   

  other_interactions                                    lat/long shouting  
0                NaN  POINT (-73.9702676472613 40.7762126854894)    False  
1                NaN  POINT (-73.9683613516225 40.7725908847499)    False  
2                NaN  POINT (-73.9722500196844 40.7742879599026)    False  
3                NaN  POINT (-73.9695063535333 40.7823507678183)    False  
4                NaN  POINT (-73.9532170504865 40.7919669739962)    False

In [51]:
# Loop through the s2018 booleans, if there's a match in the other_df, then compare cols
count = 0

# Create a variable to hold the different column names
s2018_columns = s2018_booleans.columns[7:12] # ['running', 'chasing', 'climbing', 'eating', 'foraging']
other_columns = other_df.columns[2:] # ['sitting', 'digging', 'chasing', 'jumping', 'climbing', 'eating', 'running', 'foraging']

# Append missing columns to s2018_booleans
missing_cols = list(set(other_columns).difference(s2018_columns)) # ['sitting', 'jumping', 'digging']
for col in missing_cols:
    s2018_booleans[col] = pd.Series(dtype=bool)

# Loop through both dataframes
for idx, row in s2018_booleans.iterrows():
    for idx_other, row_other in other_df.iterrows():
        
        # If the squirrel_id exists in both dataframes, check the columns
        if (row['squirrel_id'] == row_other['squirrel_id']):
            
            
            for s2018_col in s2018_columns:
                for other_col in other_columns:
                    
                    # Only interested where the s2018_col matches other_col, AND if True
                    if (s2018_col == other_col) and (other_df.loc[idx_other, other_col] == True):

                        # Update existing columns as required
                        s2018_booleans.loc[idx, s2018_col] = True

                    # Columns that are not yet in the s2018_booleans dataframe
                    if (other_col in missing_cols) and (other_df.loc[idx_other, other_col] == True):
                        s2018_booleans.loc[idx, other_col] = True
                    else:
                        s2018_booleans.loc[idx, other_col] = False
            count += 1
print(count)

285


In [52]:
s2018_booleans['digging'].value_counts()

False    248
True      37
Name: digging, dtype: int64

In [53]:
s2018_booleans['other_interactions'].value_counts()

me                            6
curious                       4
avoided people                3
runs from (me)                3
indifferent then runs from    2
                             ..
climbs                        1
runs from (goes up a tree)    1
skittish                      1
alert & aware                 1
eyes from a safe distance     1
Name: other_interactions, Length: 152, dtype: int64

## Merge both datasets

In [54]:
print(s2020_booleans.columns)
s2020_booleans.head()

Index(['park_name', 'park_id', 'squirrel_id', 'primary_fur_color',
       'highlights_in_fur_color', 'other_notes_or_observations',
       'squirrel_latitude', 'squirrel_longitude', 'shouting', 'chasing',
       'foraging', 'sitting', 'climbing', 'eating', 'other_activities',
       'digging', 'running', 'approaches', 'watching', 'indifferent',
       'runs from'],
      dtype='object')


park_name  park_id squirrel_id primary_fur_color  \
0  Fort Tryon Park        1     A-01-01              Gray   
1  Fort Tryon Park        1     A-01-02              Gray   
2  Fort Tryon Park        1     A-01-03              Gray   
3  Fort Tryon Park        1     A-01-04              Gray   
4  Fort Tryon Park        1     A-01-05              Gray   

  highlights_in_fur_color other_notes_or_observations  squirrel_latitude  \
0                   White                         NaN          40.859410   
1                   White                Looks skinny          40.859436   
2                   White                         NaN          40.859416   
3                   White                         NaN          40.859418   
4                Cinnamon               She left food          40.859493   

   squirrel_longitude  shouting  chasing  ...  sitting  climbing  eating  \
0          -73.933936     False    False  ...    False     False   False   
1          -73.933937     False    False  ...    False     False   False   
2          -73.933894     False    False  ...    False     False    True   
3          -73.933895     False    False  ...    False     False   False   
4          -73.933590     False    False  ...    False     False    True   

   other_activities  digging  running  approaches  watching  indifferent  \
0             False    False    False       False     False         True   
1             False    False    False       False     False         True   
2             False     True    False       False     False         True   
3             False    False     True       False     False         True   
4             False    False     True       False     False         True   

   runs from  
0      False  
1      False  
2      False  
3      False  
4      False  

[5 rows x 21 columns]

In [55]:
print(s2018_booleans.columns)
s2018_booleans.head()

Index(['squirrel_longitude', 'squirrel_latitude', 'squirrel_id', 'date', 'age',
       'primary_fur_color', 'highlight_fur_color', 'running', 'chasing',
       'climbing', 'eating', 'foraging', 'other_activities', 'approaches',
       'indifferent', 'runs_from', 'other_interactions', 'lat/long',
       'shouting', 'sitting', 'digging'],
      dtype='object')


squirrel_longitude  squirrel_latitude     squirrel_id        date    age  \
0          -73.970268          40.776213  13E-AM-1017-05  2018-10-17  Adult   
1          -73.968361          40.772591  11H-AM-1010-03  2018-10-10  Adult   
2          -73.972250          40.774288  11D-AM-1010-03  2018-10-10  Adult   
3          -73.969506          40.782351  20B-PM-1013-05  2018-10-13  Adult   
4          -73.953217          40.791967  36I-PM-1007-01  2018-10-07  Adult   

  primary_fur_color highlight_fur_color  running  chasing  climbing  ...  \
0              Gray            Cinnamon    False    False     False  ...   
1          Cinnamon               White    False    False     False  ...   
2              Gray            Cinnamon    False    False     False  ...   
3              Gray               White    False    False     False  ...   
4              Gray            Cinnamon    False    False     False  ...   

   foraging  other_activities approaches  indifferent  runs_from  \
0      True               NaN      False        False      False   
1      True               NaN      False         True      False   
2     False          grooming      False         True      False   
3      True               NaN      False         True      False   
4      True               NaN      False         True      False   

   other_interactions                                    lat/long shouting  \
0                 NaN  POINT (-73.9702676472613 40.7762126854894)    False   
1                 NaN  POINT (-73.9683613516225 40.7725908847499)    False   
2                 NaN  POINT (-73.9722500196844 40.7742879599026)    False   
3                 NaN  POINT (-73.9695063535333 40.7823507678183)    False   
4                 NaN  POINT (-73.9532170504865 40.7919669739962)    False   

  sitting digging  
0     NaN     NaN  
1     NaN     NaN  
2   False   False  
3     NaN     NaN  
4     NaN     NaN  

[5 rows x 21 columns]

In [56]:
# Confirm there is no overlap between the squirrel_id
ids_2020 = set(s2020_booleans['squirrel_id'].unique())
ids_2018 = set(s2018_booleans['squirrel_id'].unique())

ids_2020.intersection(ids_2018) # Since empty set, can combine the two datasets

set()

In [57]:
# Drop unused columns
s2020_clean = s2020_booleans.drop(columns=[
    'park_name', # for now, add back in later
    'park_id', # for now, add back in later
    'other_notes_or_observations',
    'other_activities'])

s2018_clean = s2018_booleans.drop(columns=[
    'date',
    'age', # for now, add back in later, once added equivalent 2020 column of NaN?
    'other_activities',
    'other_interactions',
    'lat/long'])

In [58]:
print(len(s2020_clean.columns))
s2020_clean.columns

17


Index(['squirrel_id', 'primary_fur_color', 'highlights_in_fur_color',
       'squirrel_latitude', 'squirrel_longitude', 'shouting', 'chasing',
       'foraging', 'sitting', 'climbing', 'eating', 'digging', 'running',
       'approaches', 'watching', 'indifferent', 'runs from'],
      dtype='object')

In [59]:
print(len(s2018_clean.columns))
s2018_clean.columns

16


Index(['squirrel_longitude', 'squirrel_latitude', 'squirrel_id',
       'primary_fur_color', 'highlight_fur_color', 'running', 'chasing',
       'climbing', 'eating', 'foraging', 'approaches', 'indifferent',
       'runs_from', 'shouting', 'sitting', 'digging'],
      dtype='object')

In [60]:
# Rename columns
s2020_clean = s2020_clean.rename(columns={
    'primary_fur_color': 'primary_colour',
    'highlights_in_fur_color': 'fur_highlights',
    'squirrel_longitude': 'longitude',
    'squirrel_latitude': 'latitude',
    'runs from': 'runs_from'
})

s2018_clean = s2018_clean.rename(columns={
    'squirrel_longitude': 'longitude',
    'squirrel_latitude': 'latitude',
    'primary_fur_color': 'primary_colour',
    'highlight_fur_color': 'fur_highlights'
})

In [62]:
# Rearrange columns
rearranged = [
    'squirrel_id', 'longitude', 'latitude', 'primary_colour', 'fur_highlights',
    'chasing', 'climbing', 'digging', 'eating', 'foraging', 'running', 'shouting', 'sitting',
    'approaches', 'indifferent', 'runs_from']

s2020_clean = s2020_clean[rearranged]
s2018_clean = s2018_clean[rearranged]

In [63]:
s2020_clean.head()

squirrel_id  longitude   latitude primary_colour fur_highlights  chasing  \
0     A-01-01 -73.933936  40.859410           Gray          White    False   
1     A-01-02 -73.933937  40.859436           Gray          White    False   
2     A-01-03 -73.933894  40.859416           Gray          White    False   
3     A-01-04 -73.933895  40.859418           Gray          White    False   
4     A-01-05 -73.933590  40.859493           Gray       Cinnamon    False   

   climbing  digging  eating  foraging  running  shouting  sitting  \
0     False    False   False      True    False     False    False   
1     False    False   False      True    False     False    False   
2     False     True    True     False    False     False    False   
3     False    False   False     False     True     False    False   
4     False    False    True     False     True     False    False   

   approaches  indifferent  runs_from  
0       False         True      False  
1       False         True      False  
2       False         True      False  
3       False         True      False  
4       False         True      False

In [64]:
s2018_clean.head()

squirrel_id  longitude   latitude primary_colour fur_highlights  \
0  13E-AM-1017-05 -73.970268  40.776213           Gray       Cinnamon   
1  11H-AM-1010-03 -73.968361  40.772591       Cinnamon          White   
2  11D-AM-1010-03 -73.972250  40.774288           Gray       Cinnamon   
3  20B-PM-1013-05 -73.969506  40.782351           Gray          White   
4  36I-PM-1007-01 -73.953217  40.791967           Gray       Cinnamon   

   chasing  climbing digging  eating  foraging  running shouting sitting  \
0    False     False     NaN   False      True    False    False     NaN   
1    False     False     NaN   False      True    False    False     NaN   
2    False     False   False   False     False    False    False   False   
3    False     False     NaN   False      True    False    False     NaN   
4    False     False     NaN   False      True    False    False     NaN   

   approaches  indifferent  runs_from  
0       False        False      False  
1       False         True      False  
2       False         True      False  
3       False         True      False  
4       False         True      False

In [77]:
# Append the datasets
combined_df = pd.concat([s2020_clean, s2018_clean])
combined_df.head()

squirrel_id  longitude   latitude primary_colour fur_highlights  chasing  \
0     A-01-01 -73.933936  40.859410           Gray          White    False   
1     A-01-02 -73.933937  40.859436           Gray          White    False   
2     A-01-03 -73.933894  40.859416           Gray          White    False   
3     A-01-04 -73.933895  40.859418           Gray          White    False   
4     A-01-05 -73.933590  40.859493           Gray       Cinnamon    False   

   climbing digging  eating  foraging  running shouting sitting  approaches  \
0     False   False   False      True    False    False   False       False   
1     False   False   False      True    False    False   False       False   
2     False    True    True     False    False    False   False       False   
3     False   False   False     False     True    False   False       False   
4     False   False    True     False     True    False   False       False   

   indifferent  runs_from  
0         True      False  
1         True      False  
2         True      False  
3         True      False  
4         True      False

In [79]:
combined_df.info() # need to convert some columns to bool, and check the values for digging and sitting

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2137 entries, 0 to 1944
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   squirrel_id     2137 non-null   object 
 1   longitude       2137 non-null   float64
 2   latitude        2137 non-null   float64
 3   primary_colour  2137 non-null   object 
 4   fur_highlights  2137 non-null   object 
 5   chasing         2137 non-null   bool   
 6   climbing        2137 non-null   bool   
 7   digging         477 non-null    object 
 8   eating          2137 non-null   bool   
 9   foraging        2137 non-null   bool   
 10  running         2137 non-null   bool   
 11  shouting        2137 non-null   object 
 12  sitting         477 non-null    object 
 13  approaches      2137 non-null   bool   
 14  indifferent     2137 non-null   bool   
 15  runs_from       2137 non-null   bool   
dtypes: bool(8), float64(2), object(6)
memory usage: 167.0+ KB
